# Multiscale MILP

This is a continuation of 'One Location, One Temporal Scale, One Operation, Linear Programming Example' [Example 1]. Refer Example 1 to learn the basics on how Energia models processes. 

In this example, we add another Process [Solar PV] and Storage [Li-ion Battery]. Technology choice is modeled using binaries. Moreover, the model is multiscale as the operational capacities are decision variables. 

In [1]:
from energia import *

m = Model('example2')
m.q = Periods()
m.y = 4 * m.q
m.usd = Currency()

## Resources

### Conveniently declaring components

Use m.declare(\<Object Type\>, \<list of names\>) to declare a large number of objects in one step.

In [2]:
m.declare(Resource, ['power', 'wind', 'solar'])

### Set bounds on Resource flows

Unlike wind which has bound on the total consumption, we set a daily limit on solar energy. The same bound is repeated in each quarter. The following constraints are written.

$\mathbf{cons}_{solar, network, quarter_0} \leq 100$

$\mathbf{cons}_{solar, network, quarter_1} \leq 100$

$\mathbf{cons}_{solar, network, quarter_2} \leq 100$

$\mathbf{cons}_{solar, network, quarter_3} \leq 100$

In [3]:
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

2025-10-20 23:51:13,258 [INFO] General Resource Balance for solar in (l, q): initializing constraint, adding consume(solar, l, q)
2025-10-20 23:51:13,268 [INFO] ✔ Completed in 0.0008008480072021484 seconds
2025-10-20 23:51:13,275 [INFO] Binding consume in domain (solar, l, q)
2025-10-20 23:51:13,275 [INFO] ✔ Completed in 0.00022292137145996094 seconds
2025-10-20 23:51:13,277 [INFO] General Resource Balance for wind in (l, y): initializing constraint, adding consume(wind, l, y)
2025-10-20 23:51:13,280 [INFO] ✔ Completed in 0.00020623207092285156 seconds
2025-10-20 23:51:13,281 [INFO] Binding consume in domain (wind, l, y)
2025-10-20 23:51:13,282 [INFO] ✔ Completed in 0.00016021728515625 seconds
2025-10-20 23:51:13,283 [INFO] General Resource Balance for power in (l, q): initializing constraint, adding release(power, l, q)
2025-10-20 23:51:13,284 [INFO] ✔ Completed in 0.00024771690368652344 seconds
2025-10-20 23:51:13,288 [INFO] Binding release in domain (power, l, q)
2025-10-20 23:51:13

## Operations 

### Capacity as a variable 

Here we want the optimization problem to determine the optimal capacity. Moreover, we set binaries to avoid the lower bound being adhered to if the process is not set up. 

If the bounds are meant to be compulsory limits, skip the .x 

In [4]:
m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.capacity.show()

2025-10-20 23:51:13,316 [INFO] Binding capacity in domain (wf, l, y)
2025-10-20 23:51:13,327 [INFO] ✔ Completed in 0.0038101673126220703 seconds
2025-10-20 23:51:13,376 [INFO] Binding capacity in domain (wf, l, y)
2025-10-20 23:51:13,378 [INFO] ✔ Completed in 0.00029587745666503906 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Unlike in Example 1, where the capacity was know, capacity is a variable here. 

Moreover, the expenditure associated with operating and capacitating are different

In [5]:
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49
m.operate.show(True)

2025-10-20 23:51:13,404 [INFO] Binding operate in domain (wf, l, q)
2025-10-20 23:51:13,408 [INFO] ✔ Completed in 0.00209808349609375 seconds
2025-10-20 23:51:13,412 [INFO] Mapping operate: (wf, l, q) → (wf, l, y)
2025-10-20 23:51:13,414 [INFO] ✔ Completed in 0.000s


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10

m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

2025-10-20 23:51:13,442 [INFO] Binding capacity in domain (pv, l, y)
2025-10-20 23:51:13,446 [INFO] ✔ Completed in 0.002474069595336914 seconds
2025-10-20 23:51:13,448 [INFO] Binding capacity in domain (pv, l, y)
2025-10-20 23:51:13,449 [INFO] ✔ Completed in 0.0004260540008544922 seconds
2025-10-20 23:51:13,452 [INFO] Binding operate in domain (pv, l, q)
2025-10-20 23:51:13,454 [INFO] ✔ Completed in 0.0003960132598876953 seconds
2025-10-20 23:51:13,456 [INFO] Mapping operate: (pv, l, q) → (pv, l, y)
2025-10-20 23:51:13,457 [INFO] ✔ Completed in 0.000s


### Storage Operation

energia now allows storing to require the use of other resources, example power for hydrogen cryogenic storage. 

Provide an equation similar to Process, in this case the basis is the stored resource 
If no other resource is provided, it is assumed to be the charging/discharging efficiency

Note that the following are created internally: 
1. auxilary resource  with name resource.stored 
2. charging and discharging processes as storage.charge and storage.discharge 

The parameters for each of these can be set individually, thus allowing for a wide range of modeling approaches 

In [7]:
m.lii = Storage()

m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10

# m.lii.capacity >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432

m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.discharge.capacity <= 100

2025-10-20 23:51:13,474 [INFO] Binding invcapacity in domain (power.lii, l, y)
2025-10-20 23:51:13,480 [INFO] ✔ Completed in 0.0002911090850830078 seconds
2025-10-20 23:51:13,482 [INFO] Binding invcapacity in domain (power.lii, l, y)
2025-10-20 23:51:13,483 [INFO] ✔ Completed in 0.0002601146697998047 seconds
2025-10-20 23:51:13,485 [INFO] General Resource Balance for power.lii in (l, y): initializing constraint, adding inventory(power.lii, l, y)
2025-10-20 23:51:13,488 [INFO] Binding capacity in domain (lii.charge, l, y)
2025-10-20 23:51:13,489 [INFO] ✔ Completed in 0.00017404556274414062 seconds
2025-10-20 23:51:13,491 [INFO] Binding capacity in domain (lii.discharge, l, y)
2025-10-20 23:51:13,492 [INFO] ✔ Completed in 0.00016999244689941406 seconds


## Locating Operations

Operations can be located as 

operation.locate(\<list of locations\>)

or 

m.location.operations(\<list of operations\>)

They both do the same thing 

In [8]:
m.pem = Process()

m.pem(-m.power) == m.solar 

In [9]:
m.pv.locate(m.network)
m.network.locate(m.wf, m.lii)

2025-10-20 23:51:13,554 [INFO] General Resource Balance for power in (l, q): adding produce(power, l, q, operate, pv)
2025-10-20 23:51:13,561 [INFO] ✔ Completed in 0.004060983657836914 seconds
2025-10-20 23:51:13,566 [INFO] General Resource Balance for solar in (l, q): adding expend(solar, l, q, operate, pv)
2025-10-20 23:51:13,568 [INFO] ✔ Completed in 0.00028204917907714844 seconds
2025-10-20 23:51:13,570 [INFO] General Resource Balance for power in (l, q): adding produce(power, l, q, operate, wf)
2025-10-20 23:51:13,571 [INFO] ✔ Completed in 0.0003428459167480469 seconds
2025-10-20 23:51:13,573 [INFO] General Resource Balance for wind in (l, y): adding expend(wind, l, y, operate, wf)
2025-10-20 23:51:13,574 [INFO] ✔ Completed in 0.00024509429931640625 seconds
2025-10-20 23:51:13,576 [INFO] Assuming  power.lii inventory capacity is unbounded in (l, y)
2025-10-20 23:51:13,577 [INFO] Assuming inventory of power.lii is bound by inventory capacity in (l, y)
2025-10-20 23:51:13,578 [INFO]

## Inventory Balance

Inventory is passed on from one time period (t - 1) to the next (t) and hence features in the general resource balance for resource.stored 

In [10]:
m.inventory.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Optimize!

In [11]:
m.usd.spend.opt()

2025-10-20 23:51:13,694 [INFO] Mapping spend: (usd, l, y, capacity, wf) → (usd, l, y)
2025-10-20 23:51:13,701 [INFO] ✔ Completed in 0.000s
2025-10-20 23:51:13,710 [INFO] Mapping spend: (usd, l, y, operate, wf) → (usd, l, y)
2025-10-20 23:51:13,715 [INFO] ✔ Completed in 0.000s
2025-10-20 23:51:13,716 [INFO] Mapping spend: (usd, l, y, capacity, pv) → (usd, l, y)
2025-10-20 23:51:13,717 [INFO] ✔ Completed in 0.000s
2025-10-20 23:51:13,718 [INFO] Mapping spend: (usd, l, y, operate, pv) → (usd, l, y)
2025-10-20 23:51:13,719 [INFO] ✔ Completed in 0.000s
2025-10-20 23:51:13,721 [INFO] Mapping spend: (usd, l, y, invcapacity, power.lii) → (usd, l, y)
2025-10-20 23:51:13,722 [INFO] ✔ Completed in 0.000s
2025-10-20 23:51:13,723 [INFO] Mapping spend: (usd, l, y, inventory, power.lii) → (usd, l, y)
2025-10-20 23:51:13,725 [INFO] ✔ Completed in 0.000s
2025-10-20 23:51:13,726 [INFO] Generating Program(example2).mps
2025-10-20 23:51:13,738 [INFO] Creating gurobi model for Program(example2)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-12-16
Read MPS format model from file Program(example2).mps
Reading time = 0.00 seconds
PROGRAM(EXAMPLE2): 85 rows, 78 columns, 198 nonzeros


2025-10-20 23:51:13,752 [INFO] Optimizing Program(example2) using gurobi


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i7-6567U CPU @ 3.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 85 rows, 78 columns and 198 nonzeros
Model fingerprint: 0x2d658939
Variable types: 75 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 4e+02]
Presolve removed 70 rows and 63 columns
Presolve time: 0.00s
Presolved: 15 rows, 15 columns, 44 nonzeros
Variable types: 15 continuous, 0 integer (0 binary)

Root relaxation: objective 3.006497e+08, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3.006497e+08 3.0065e+08  0.00%     -    0s

Explored 1 

2025-10-20 23:51:13,788 [INFO] Solution found. Use .sol() to display it
2025-10-20 23:51:13,790 [INFO] Creating Solution object, check.solution


## Solution

### Inventory Profiles

The inventory maintained in each time period is:

In [12]:
m.inventory.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount charged into inventory is:

In [13]:
m.produce(m.power.lii, m.lii.charge.operate, m.q).sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount discharged from inventory is:

In [14]:
m.produce(m.power, m.lii.discharge.operate, m.q).sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Integer Decisions 

All the operations are setup in this case

In [15]:
m.capacity.reporting.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [16]:
m.capacity.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>